In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import pandas as pd

<h1># 01 - Ergebnisdaten: Parsen der Ergebnisdaten - New York (2007 - 2019)</h1>
<hr>
<p><b>Hinweis:</b> Dateifpade sind absolut angegeben und müssen entsprechend der eigenen Verzeichnisstruktur angepasst werden!</p>
<hr>

In [ ]:
# Hilfsfunktion: Prüfung, ob ein HTML existiert um Abbrüchen beim parsen zu vermeiden.
def check_if_element_exists(web_driver, xpath):
    try:
        web_driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return ""
    return web_driver.find_element_by_xpath(xpath).text

In [ ]:
# Treiber laden
driver = webdriver.Firefox()

In [ ]:
for year in range(2007, 2020):
    # Seite laden 
    url = "https://results.nyrr.org/races"
    driver.get(url)
    time.sleep(10)

    # Filter nach Marathon-Läufen
    driver.find_element_by_xpath("//optgroup[@label='Popular Races:']/option[@label='Marathon']").click()
    time.sleep(5)


    # Leeres Dataframe anlegen
    df_results = pd.DataFrame(pd.DataFrame(
    columns=[
        'Jahr','Ges','Start_Time_of_day'
        ,'Name'
        ,'Runner_Number'
        ,'FK_Time'
        ,'Pace_per_Mile'
        ,'Place_Overall'
        ,'Place_Gender'
        ,'Age_Group'
        ,'Place_Age_Group'
        ,'Place_Age_Graded'
        ,'Time_Age_Graded'
        ,'Gun_Time'
        ,'5K_Time'
        ,'10K_Time'
        ,'15K_Time'
        ,'20K_Time'
        ,'HK_Time'
        ,'25K_Time'
        ,'30K_Time'
        ,'35K_Time'
        ,'40K_Time'
        ])
    ,dtype=str)

    # Ergebnisse auswaehlen und anzeigen

    # Jahr
    if year >= 2014:
        link_text =  "TCS New York City Marathon " + str(year)
    elif year in [2011,2013]:
        link_text = "ING New York City Marathon " + str(year)
    elif year <= 2010:
        link_text = str(year) + " ING New York City Marathon"
    
    xpath_select_result_list = "//div[text()='" + link_text + "']/parent::a[1]"
    link_to_event = driver.find_element_by_xpath(xpath_select_result_list).get_attribute("href")
    driver.get(link_to_event)
    time.sleep(10)
    
    for ges in ['M', 'W']:

        # Geschlecht - 1) Dropdown anklicken
        driver.find_element_by_xpath("//div[@class='dropdown dropdown-main short-filter']/a[1]").click()
        time.sleep(3)
        # Geschlecht - 2) Geschlecht auswählen
        ges_filter = 'Men' if ges == 'M' else 'Women'
        driver.find_element_by_xpath("//div[@class='dropdown dropdown-main short-filter open']/div[@class='dropdown-menu']//a[text()='" + ges_filter + "']").click()
        time.sleep(5)

        # Uber die ersten 10 Athleten iterieren (Header uebersprungen)
        for i in range(1,11):
            # Ein Athlet aufrufen        
            xpath_link = "//div[@ng-show='eventFinishers.length > 0']/div[" + str(i) + "]/div[1]/div[1]"
            driver.find_element_by_xpath(xpath_link).click()
            time.sleep(4)
            xpath_content_box_container_res = "//div[@class='content-box-container']/div[1]"
            xpath_content_box_container_res_splits = xpath_content_box_container_res + "/div[6]/div[3]"
            
            # Auf der naechsten Seite die Daten abziehen
            
            # Slice-Wert für die Extraktion der Startnummer notwending
            bib_slice = -3 if ges == 'W' else -2
            
            result_temp = {
                'Jahr' : year
                ,'Ges' : ges
                
                #,'Start_Time_of_day' : driver.find_element_by_xpath("//div[@class='event-desc ng-scope no-left-padding']/p[1]").text[-6:] # works till 2017
                #,'Start_Time_of_day' : driver.find_element_by_xpath("//h1[text()='" + link_text +"']/parent::div[1]/p[1]").text[-6:] # works up 2017

                ,'Start_Time_of_day' : driver.find_element_by_xpath("//div[@class='event-desc ng-scope no-left-padding']/p[1]").text[-6:] if year < 2017 
                            else driver.find_element_by_xpath("//h1[text()='" + link_text +"']/parent::div[1]/p[1]").text[-6:]

                ,'Name' : driver.find_element_by_xpath(xpath_content_box_container_res + "/div[1]/div[1]//div[@class='main-title ng-binding']").text
                ,'Runner_Number' : driver.find_element_by_xpath(xpath_content_box_container_res + "/div[1]/div[3]/div[1]/div[1]").text[bib_slice:]

                ,'FK_Time' : driver.find_element_by_xpath(xpath_content_box_container_res + "/div[2]/div[1]/span[1]").text
                ,'Pace_per_Mile' : driver.find_element_by_xpath(xpath_content_box_container_res + "/div[2]/div[2]/span[1]").text
                ,'Place_Overall' : driver.find_element_by_xpath(xpath_content_box_container_res + "/div[2]/div[3]/span[1]").text                

                ,'Place_Gender' : driver.find_element_by_xpath(xpath_content_box_container_res + "/div[3]/div[1]/span[1]").text
                ,'Age_Group' : driver.find_element_by_xpath(xpath_content_box_container_res + "/div[3]/div[2]/label[1]/span[1]").text      
                ,'Place_Age_Group' : driver.find_element_by_xpath(xpath_content_box_container_res + "/div[3]/div[2]/span[1]").text
                
                ,'Place_Age_Graded' : driver.find_element_by_xpath(xpath_content_box_container_res + "/div[4]/div[1]/span[1]").text    
                ,'Time_Age_Graded' : driver.find_element_by_xpath(xpath_content_box_container_res + "/div[4]/div[2]/span[1]").text    

                ,'Gun_Time' : driver.find_element_by_xpath(xpath_content_box_container_res + "/div[5]/div[1]/span[1]").text                                       
                
                ,'5K_Time' : check_if_element_exists(driver, xpath_content_box_container_res_splits + "//label[text()='5K']/parent::div[1]/parent::div[1]/div[2]/span[1]")

                ,'10K_Time' : check_if_element_exists(driver, xpath_content_box_container_res_splits + "//label[text()='10K']/parent::div[1]/parent::div[1]/div[2]/span[1]")

                ,'15K_Time' : check_if_element_exists(driver, xpath_content_box_container_res_splits + "//label[text()='15K']/parent::div[1]/parent::div[1]/div[2]/span[1]")

                ,'20K_Time' : check_if_element_exists(driver, xpath_content_box_container_res_splits + "//label[text()='20K']/parent::div[1]/parent::div[1]/div[2]/span[1]")

                ,'HK_Time' : check_if_element_exists(driver, xpath_content_box_container_res_splits + "//label[text()='HALF']/parent::div[1]/parent::div[1]/div[2]/span[1]")

                ,'25K_Time' : check_if_element_exists(driver, xpath_content_box_container_res_splits + "//label[text()='25K']/parent::div[1]/parent::div[1]/div[2]/span[1]")
          
                ,'30K_Time' : check_if_element_exists(driver, xpath_content_box_container_res_splits + "//label[text()='30K']/parent::div[1]/parent::div[1]/div[2]/span[1]")
                
                ,'35K_Time' : check_if_element_exists(driver, xpath_content_box_container_res_splits + "//label[text()='35K']/parent::div[1]/parent::div[1]/div[2]/span[1]")
                
                ,'40K_Time' : check_if_element_exists(driver, xpath_content_box_container_res_splits + "//label[text()='40K']/parent::div[1]/parent::div[1]/div[2]/span[1]")
                
            }
            df_results = df_results.append(result_temp, ignore_index=True)

            # Zurueck zu der Athletenliste
            driver.find_element_by_xpath("//div[@class='top-buttons']/a[@class='back-btn']").click()
            time.sleep(3)

    
    # Ergebnisse anzeigen
    # print(df_results.head())

    # Daten in die CSV Exportieren
    file_path = '/home/paul/python_projects/masterthesis/data/NYC/'
    file_name = 'daten_wmm_nyc_' + str(year) + '.csv'
    df_results.to_csv(file_path+file_name, sep=';', index=False)

In [ ]:

# Treiber schliessen
driver.close()